# MICCAI 2023 Tutorial - Part I:
## Launch a containerized medical imaging application
This Notebook is designed to reproduce the computational experiments of a [published scientific paper](https://hal.science/hal-04006057) on deep learning for **Bra**in **T**umor **S**egmentation (**BraTS**).

<img src="imgs/BraTS-Results.png" alt="BraTS-Results" height="250" title="Results from Brain Tumor Segmentation"/>

It uses methods from the [VIP client for Python](https://github.com/virtual-imaging-platform/VIP-python-client).

> __N.B.__: This tutorial is designed as a team work with other attendees. __Please wait for the speakers' instructions when required__ by the *italic-style cell comments*.

---
## Useful libraries

**Install step**: uncomment & run the following cell *only* if you are using *Google Colab* (not *Binder*)

In [ ]:
# Remove the hashtag (#) only
#! pip install vip-client girder-client

**Importation step**

In [ ]:
# Builtins
import json
from pathlib import *
from random import choice
# Installed
from vip_client.classes import VipLauncher
from vip_client.classes import VipLoader

---
## Prepare your executions

Handshake with VIP using your **VIP API key** (get it from the [VIP portal](https://vip.creatis.insa-lyon.fr/))

In [ ]:
VipLauncher.init(api_key="VIP_API_KEY"); # Paste your VIP API key here

Show the applications that will be used in this tutorial 

In [ ]:
VipLauncher.show_pipeline("BraTS")

Application `BraTSPipeline` is declined in two different versions. You will be able to launch one of them in a few cells.

In [ ]:
pipelines = {
    "v181": "BraTSPipeline/1.8.1_cvmfs", 
    "v190": "BraTSPipeline/1.9.0_cvmfs"
}
# Flip a coin to choose your version
my_version = choice(list(pipelines))
print("My version is:", my_version)

Show the inputs requiered by such application

In [ ]:
VipLauncher.show_pipeline(pipelines[my_version])

Explore the dataset

In [ ]:
# The data you need is stored in a folder on Vip Servers 
input_dir = PurePosixPath("/vip/EGI tutorial (group)/inputs")
# This folder contains data from several subjects
subjects = VipLoader.list_dir(input_dir)
print("\n".join(subjects))

Build and print your inputs

In [ ]:
# Each subject contains 4 inputs files for the BraTS pipeline
input_settings = {
    "t1Image": [(input_dir / subject / "T1.nii.gz") for subject in subjects],
    "t1ceImage": [(input_dir / subject / "T1GD.nii.gz") for subject in subjects],
    "t2Image": [(input_dir / subject / "T2.nii.gz") for subject in subjects],
    "flImage": [(input_dir / subject / "T2-FLAIR.nii.gz") for subject in subjects],
    "appliOutputDir": subjects,
    "brainTumor": "1",
    "skullStrip": "1",
}

# Print your inputs
print("input_settings =",
    json.dumps(indent=2, 
               obj={ key: [str(v) for v in value] if isinstance(value, list) else str(value) for key, value in input_settings.items() } )
)

---
## Launch the BraTS pipeline

*To control the size of your dataset, the pipeline will be launched in __teams__.*
*Please modify the following cell according to speakers' instructions:*
> *Put your __team number__ instead of "X"*

In [ ]:
# PUT YOUR TEAM NUMBER
my_team = "miccai-team-1"

if my_team == "miccai-team-X":
    raise ValueError("Please precise your team number.")

Launch the application

*Please __do not__ execute the following cell __more than once__ to avoid overloading the servers*

In [ ]:
VipLauncher(
    output_dir = input_dir.parent / "outputs" / my_team / my_version,
    pipeline_id = pipelines[my_version],
    input_settings = input_settings
).run_session()

You should be invited to monitor your executions on the [VIP portal](https://vip.creatis.insa-lyon.fr).

When most executions are over for your tutorial team, you may proceed to **[Part II](2-analyze-outputs.ipynb)**.

---